# EDA
* Remove dt<12 and dt = 24
* Remove chid whose top1 value get 0
* Split 22 > dt > 11 as train set, dt > 22 as test set

# Modeling
* Only using rank column as feature 
* Modeling TOP1 by CatBoostClassifer
* Train score: 0.558, Test score: 0.426

In [1]:
pip install catboost

     |████████████████████████████████| 76.3 MB 29 kB/s 


In [2]:
from catboost import Pool, CatBoostClassifier
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 55)
pd.set_option('display.max_rows', 200)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
import os
os.chdir(path='./gdrive/MyDrive/SideProject/ESun2021WinterGame/notebook')

In [5]:
df = pd.read_feather('../data/rank_data.feather')

In [6]:
df.shape

(72024, 594)

In [7]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72024 entries, 0 to 72023
Columns: 594 entries, index to shop_tag_27_rank_t-11
dtypes: float64(587), int64(1), object(6)
memory usage: 326.4+ MB


None

In [8]:
df.head()

,index,dt,shop_tag_15_rank,shop_tag_19_rank,shop_tag_2_rank,shop_tag_26_rank,shop_tag_34_rank,shop_tag_35_rank,shop_tag_36_rank,shop_tag_37_rank,shop_tag_45_rank,shop_tag_48_rank,shop_tag_50_rank,shop_tag_6_rank,shop_tag_15_rank_t-1,shop_tag_19_rank_t-1,shop_tag_2_rank_t-1,shop_tag_26_rank_t-1,shop_tag_34_rank_t-1,shop_tag_35_rank_t-1,shop_tag_36_rank_t-1,shop_tag_37_rank_t-1,shop_tag_45_rank_t-1,shop_tag_48_rank_t-1,shop_tag_50_rank_t-1,shop_tag_6_rank_t-1,shop_tag_15_rank_t-2,...,shop_tag_1_rank_t-9,shop_tag_1_rank_t-10,shop_tag_1_rank_t-11,shop_tag_5_rank,shop_tag_5_rank_t-1,shop_tag_5_rank_t-2,shop_tag_5_rank_t-3,shop_tag_5_rank_t-4,shop_tag_5_rank_t-5,shop_tag_5_rank_t-6,shop_tag_5_rank_t-7,shop_tag_5_rank_t-8,shop_tag_5_rank_t-9,shop_tag_5_rank_t-10,shop_tag_5_rank_t-11,shop_tag_27_rank,shop_tag_27_rank_t-1,shop_tag_27_rank_t-2,shop_tag_27_rank_t-3,shop_tag_27_rank_t-4,shop_tag_27_rank_t-5,shop_tag_27_rank_t-6,shop_tag_27_rank_t-7,shop_tag_27_rank_t-8,shop_tag_27_rank_t-9,shop_tag_27_rank_t-10,shop_tag_27_rank_t-11
0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,48.0,1.0,49.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,49.0,48.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,48.0,1.0,49.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,48.0,47.0,1.0,49.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,49.0,48.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,49.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,48.0,47.0,1.0,49.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,49.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,49.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.describe()

,index,shop_tag_15_rank,shop_tag_19_rank,shop_tag_2_rank,shop_tag_26_rank,shop_tag_34_rank,shop_tag_35_rank,shop_tag_36_rank,shop_tag_37_rank,shop_tag_45_rank,shop_tag_48_rank,shop_tag_50_rank,shop_tag_6_rank,shop_tag_15_rank_t-1,shop_tag_19_rank_t-1,shop_tag_2_rank_t-1,shop_tag_26_rank_t-1,shop_tag_34_rank_t-1,shop_tag_35_rank_t-1,shop_tag_36_rank_t-1,shop_tag_37_rank_t-1,shop_tag_45_rank_t-1,shop_tag_48_rank_t-1,shop_tag_50_rank_t-1,shop_tag_6_rank_t-1,shop_tag_15_rank_t-2,shop_tag_19_rank_t-2,...,shop_tag_1_rank_t-9,shop_tag_1_rank_t-10,shop_tag_1_rank_t-11,shop_tag_5_rank,shop_tag_5_rank_t-1,shop_tag_5_rank_t-2,shop_tag_5_rank_t-3,shop_tag_5_rank_t-4,shop_tag_5_rank_t-5,shop_tag_5_rank_t-6,shop_tag_5_rank_t-7,shop_tag_5_rank_t-8,shop_tag_5_rank_t-9,shop_tag_5_rank_t-10,shop_tag_5_rank_t-11,shop_tag_27_rank,shop_tag_27_rank_t-1,shop_tag_27_rank_t-2,shop_tag_27_rank_t-3,shop_tag_27_rank_t-4,shop_tag_27_rank_t-5,shop_tag_27_rank_t-6,shop_tag_27_rank_t-7,shop_tag_27_rank_t-8,shop_tag_27_rank_t-9,shop_tag_27_rank_t-10,shop_tag_27_rank_t-11
count,72024.000000,51606.000000,40512.000000,46679.000000,25342.000000,16737.000000,10958.000000,37063.000000,57424.000000,40113.000000,41145.000000,54267.000000,26378.000000,49457.000000,38837.000000,44740.000000,24279.000000,16049.000000,10501.000000,35525.000000,55042.000000,38446.000000,39434.000000,52002.000000,25278.000000,47308.000000,37141.000000,...,724.000000,677.000000,630.000000,7543.000000,7233.000000,6918.000000,6603.000000,6293.000000,5981.000000,5665.000000,5348.000000,5034.000000,4719.000000,4400.000000,4092.000000,462.000000,442.000000,423.000000,403.000000,384.000000,364.000000,345.000000,325.000000,306.000000,286.000000,267.000000,248.000000
mean,11.500000,16.840619,9.282632,14.088948,7.698051,5.242935,6.823508,21.590535,25.949272,22.375589,16.495346,19.930437,9.447077,16.754676,9.291912,14.077962,7.738251,5.416661,6.624988,21.586517,25.937157,22.428159,16.521225,19.890639,9.422739,16.694322,9.254355,...,9.857735,10.042836,9.941270,9.102081,8.915664,8.776670,8.665304,8.582870,8.486206,8.366461,8.195587,8.072507,7.961433,7.823636,7.671799,4.196970,4.341629,4.274232,4.320099,4.484375,4.675824,4.736232,4.815385,5.052288,5.335664,5.460674,5.616935
std,6.922235,22.007693,17.888229,21.060359,16.313050,13.662852,15.602326,22.977894,23.708626,23.321912,22.057084,22.972864,17.999314,21.973920,17.895040,21.052537,16.356468,13.913547,15.364539,22.977879,23.709063,23.325693,22.068692,22.965564,17.978849,21.953276,17.860960,...,18.775309,18.928148,18.840593,17.297908,17.147729,17.036236,16.947682,16.879684,16.798382,16.693128,16.544376,16.441665,16.348168,16.233556,16.096250,11.759805,12.003346,11.894699,11.969042,12.238908,12.543450,12.623229,12.729246,13.082984,13.488628,13.643740,13.856788
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,5.750000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,11.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,44.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,44.000000,1.000000,1.000000,1.000000,1.000

In [10]:
df_col = df.columns.tolist()
df.columns.tolist()

['index',
 'dt',
 'shop_tag_15_rank',
 'shop_tag_19_rank',
 'shop_tag_2_rank',
 'shop_tag_26_rank',
 'shop_tag_34_rank',
 'shop_tag_35_rank',
 'shop_tag_36_rank',
 'shop_tag_37_rank',
 'shop_tag_45_rank',
 'shop_tag_48_rank',
 'shop_tag_50_rank',
 'shop_tag_6_rank',
 'shop_tag_15_rank_t-1',
 'shop_tag_19_rank_t-1',
 'shop_tag_2_rank_t-1',
 'shop_tag_26_rank_t-1',
 'shop_tag_34_rank_t-1',
 'shop_tag_35_rank_t-1',
 'shop_tag_36_rank_t-1',
 'shop_tag_37_rank_t-1',
 'shop_tag_45_rank_t-1',
 'shop_tag_48_rank_t-1',
 'shop_tag_50_rank_t-1',
 'shop_tag_6_rank_t-1',
 'shop_tag_15_rank_t-2',
 'shop_tag_19_rank_t-2',
 'shop_tag_2_rank_t-2',
 'shop_tag_26_rank_t-2',
 'shop_tag_34_rank_t-2',
 'shop_tag_35_rank_t-2',
 'shop_tag_36_rank_t-2',
 'shop_tag_37_rank_t-2',
 'shop_tag_45_rank_t-2',
 'shop_tag_48_rank_t-2',
 'shop_tag_50_rank_t-2',
 'shop_tag_6_rank_t-2',
 'shop_tag_15_rank_t-3',
 'shop_tag_19_rank_t-3',
 'shop_tag_2_rank_t-3',
 'shop_tag_26_rank_t-3',
 'shop_tag_34_rank_t-3',
 'shop_tag_35

In [11]:
y_col = ['top1', 'top2', 'top3', 'top4', 'top5']

In [12]:
x_col = list(set(df_col).difference(y_col))
[x_col.remove(i) for i in ['dt', 'chid', 'index']]
display(x_col)

['shop_tag_8_rank_t-8',
 'shop_tag_3_rank_t-3',
 'shop_tag_11_rank_t-10',
 'shop_tag_39_rank_t-11',
 'shop_tag_40_rank_t-6',
 'shop_tag_11_rank_t-8',
 'shop_tag_46_rank_t-8',
 'shop_tag_5_rank_t-4',
 'shop_tag_23_rank_t-9',
 'masts',
 'shop_tag_42_rank_t-10',
 'shop_tag_16_rank_t-2',
 'shop_tag_28_rank_t-7',
 'shop_tag_36_rank_t-10',
 'shop_tag_19_rank_t-1',
 'shop_tag_15_rank',
 'shop_tag_42_rank_t-11',
 'shop_tag_48_rank_t-4',
 'shop_tag_33_rank_t-5',
 'shop_tag_10_rank_t-11',
 'shop_tag_32_rank_t-9',
 'shop_tag_16_rank_t-4',
 'shop_tag_19_rank_t-11',
 'shop_tag_32_rank_t-1',
 'shop_tag_14_rank_t-11',
 'shop_tag_39_rank_t-2',
 'shop_tag_22_rank_t-6',
 'shop_tag_33_rank_t-10',
 'shop_tag_41_rank_t-8',
 'shop_tag_46_rank_t-11',
 'shop_tag_45_rank_t-10',
 'shop_tag_7_rank_t-1',
 'shop_tag_45_rank_t-1',
 'shop_tag_45_rank_t-4',
 'shop_tag_47_rank_t-2',
 'shop_tag_19_rank_t-4',
 'shop_tag_40_rank_t-9',
 'shop_tag_23_rank_t-11',
 'shop_tag_12_rank_t-9',
 'shop_tag_46_rank_t-7',
 'shop_tag_

In [13]:
category_x_col = ['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'gender_code', 'age', 'primary_card']
# masts: 婚姻狀況; trdtp: 行業別; naty: 國籍; poscd: 職位別; cuorg: 客戶來源

In [14]:
numerical_x_col = list(set(x_col).difference(category_x_col))
display(numerical_x_col)

['shop_tag_8_rank_t-8',
 'shop_tag_3_rank_t-3',
 'shop_tag_11_rank_t-10',
 'shop_tag_39_rank_t-11',
 'shop_tag_11_rank_t-8',
 'shop_tag_46_rank_t-8',
 'shop_tag_40_rank_t-6',
 'shop_tag_5_rank_t-4',
 'shop_tag_23_rank_t-9',
 'shop_tag_42_rank_t-10',
 'shop_tag_16_rank_t-2',
 'shop_tag_28_rank_t-7',
 'shop_tag_36_rank_t-10',
 'shop_tag_19_rank_t-1',
 'shop_tag_15_rank',
 'shop_tag_42_rank_t-11',
 'shop_tag_48_rank_t-4',
 'shop_tag_33_rank_t-5',
 'shop_tag_10_rank_t-11',
 'shop_tag_32_rank_t-9',
 'shop_tag_16_rank_t-4',
 'shop_tag_19_rank_t-11',
 'shop_tag_32_rank_t-1',
 'shop_tag_14_rank_t-11',
 'shop_tag_39_rank_t-2',
 'shop_tag_22_rank_t-6',
 'shop_tag_33_rank_t-10',
 'shop_tag_41_rank_t-8',
 'shop_tag_46_rank_t-11',
 'shop_tag_45_rank_t-10',
 'shop_tag_7_rank_t-1',
 'shop_tag_45_rank_t-1',
 'shop_tag_45_rank_t-4',
 'shop_tag_47_rank_t-2',
 'shop_tag_19_rank_t-4',
 'shop_tag_40_rank_t-9',
 'shop_tag_23_rank_t-11',
 'shop_tag_12_rank_t-9',
 'shop_tag_46_rank_t-7',
 'shop_tag_33_rank_t-

In [15]:
rank_col = list(set(numerical_x_col).difference(['slam', 'poscd']))
display(rank_col)

['shop_tag_8_rank_t-8',
 'shop_tag_3_rank_t-3',
 'shop_tag_11_rank_t-10',
 'shop_tag_39_rank_t-11',
 'shop_tag_40_rank_t-6',
 'shop_tag_46_rank_t-8',
 'shop_tag_11_rank_t-8',
 'shop_tag_5_rank_t-4',
 'shop_tag_23_rank_t-9',
 'shop_tag_42_rank_t-10',
 'shop_tag_16_rank_t-2',
 'shop_tag_28_rank_t-7',
 'shop_tag_36_rank_t-10',
 'shop_tag_19_rank_t-1',
 'shop_tag_15_rank',
 'shop_tag_42_rank_t-11',
 'shop_tag_48_rank_t-4',
 'shop_tag_33_rank_t-5',
 'shop_tag_10_rank_t-11',
 'shop_tag_32_rank_t-9',
 'shop_tag_16_rank_t-4',
 'shop_tag_19_rank_t-11',
 'shop_tag_32_rank_t-1',
 'shop_tag_14_rank_t-11',
 'shop_tag_39_rank_t-2',
 'shop_tag_22_rank_t-6',
 'shop_tag_33_rank_t-10',
 'shop_tag_41_rank_t-8',
 'shop_tag_46_rank_t-11',
 'shop_tag_45_rank_t-10',
 'shop_tag_7_rank_t-1',
 'shop_tag_45_rank_t-1',
 'shop_tag_45_rank_t-4',
 'shop_tag_47_rank_t-2',
 'shop_tag_19_rank_t-4',
 'shop_tag_40_rank_t-9',
 'shop_tag_23_rank_t-11',
 'shop_tag_12_rank_t-9',
 'shop_tag_46_rank_t-7',
 'shop_tag_33_rank_t-

# TOP1 Modeling

### Check the percentage of Null in column befor imputation

In [16]:
display(df.isnull().sum()/df.shape[0]*100)

index                     0.000000
dt                        0.000000
shop_tag_15_rank         28.348884
shop_tag_19_rank         43.752083
shop_tag_2_rank          35.189659
                           ...    
shop_tag_27_rank_t-7     99.548762
shop_tag_27_rank_t-8     99.575142
shop_tag_27_rank_t-9     99.602910
shop_tag_27_rank_t-10    99.629290
shop_tag_27_rank_t-11    99.655670
Length: 594, dtype: float64

In [17]:
# 'gender_code' and 'age' have missing value 
df[category_x_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72024 entries, 0 to 72023
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   masts         72024 non-null  float64
 1   educd         72024 non-null  float64
 2   trdtp         72024 non-null  float64
 3   naty          72024 non-null  float64
 4   poscd         72024 non-null  float64
 5   cuorg         72024 non-null  float64
 6   gender_code   71256 non-null  float64
 7   age           71256 non-null  float64
 8   primary_card  72024 non-null  float64
dtypes: float64(9)
memory usage: 4.9 MB


### Fillna in rank_col 

In [18]:
display('Before Imputation:', df[rank_col].isnull().sum()/df.shape[0]*100)
df[rank_col] = df[rank_col].fillna(1.0)
display('After Imputation:', df[rank_col].isnull().sum()/df.shape[0]*100)

'Before Imputation:'

shop_tag_8_rank_t-8      94.249139
shop_tag_3_rank_t-3      88.288626
shop_tag_11_rank_t-10    82.382261
shop_tag_39_rank_t-11    79.000056
shop_tag_40_rank_t-6     87.444463
                           ...    
shop_tag_2_rank_t-3      43.286960
shop_tag_40_rank_t-5     86.754415
shop_tag_39_rank_t-8     74.164167
shop_tag_27_rank_t-5     99.494613
shop_tag_34_rank_t-4     80.609241
Length: 576, dtype: float64

'After Imputation:'

shop_tag_8_rank_t-8      0.0
shop_tag_3_rank_t-3      0.0
shop_tag_11_rank_t-10    0.0
shop_tag_39_rank_t-11    0.0
shop_tag_40_rank_t-6     0.0
                        ... 
shop_tag_2_rank_t-3      0.0
shop_tag_40_rank_t-5     0.0
shop_tag_39_rank_t-8     0.0
shop_tag_27_rank_t-5     0.0
shop_tag_34_rank_t-4     0.0
Length: 576, dtype: float64

### Display the top1 2 top5 before shifting

In [19]:
display(df.loc[:,['dt', 'top1', 'top2', 'top3', 'top4', 'top5']])

,dt,top1,top2,top3,top4,top5
0,1.0,37,50,45,45,6
1,2.0,34,37,45,45,6
2,3.0,34,50,37,45,45
3,4.0,37,45,45,6,50
4,5.0,37,45,45,6,50
...,...,...,...,...,...,...
72019,20.0,2,15,12,25,37
72020,21.0,24,2,37,10,12
72021,22.0,2,19,10,15,12
72022,23.0,2,15,25,12,37


### Shift the y_col to achieve using x[dt(t-1)] predict y[dt(t)]

In [20]:
df[['top1','top2', 'top3', 'top4', 'top5']] = df[['top1','top2', 'top3', 'top4', 'top5']].shift(periods=-1)

### Remove dt = 24 and df < 12  for training

In [21]:
# df.loc[df['dt'] == '24.0', ['top1','top2', 'top3', 'top4', 'top5']] = np.nan
df = df[(df['dt'].astype('float') < 24) & (df['dt'].astype('float') > 11)]

### Check the output after shifting

In [22]:
display(df.loc[:,['dt', 'top1', 'top2', 'top3', 'top4', 'top5']])

,dt,top1,top2,top3,top4,top5
11,12.0,37,45,45,6,50
12,13.0,34,50,37,45,2
13,14.0,34,37,45,45,6
14,15.0,37,45,45,6,50
15,16.0,37,50,45,45,6
...,...,...,...,...,...,...
72018,19.0,2,15,12,25,37
72019,20.0,24,2,37,10,12
72020,21.0,2,19,10,15,12
72021,22.0,2,15,25,12,37


### Remove the chid whose top1 values get 0

In [23]:
df = df[~df.chid.isin(list(df[df['top1']=='0'].chid.unique()))].reset_index(drop=True)
display(df.shape)

(22476, 594)

### Split df to traib/test set by dt = 22

In [24]:
test_df = df[(df['dt'].astype('float') >= 22)]
train_df = df.iloc[list(set(df.index).difference(test_df.index)),:]
display('Train set shape: ', train_df.shape, 'Test set shape: ', test_df.shape)
display(test_df)

'Train set shape: '

(18730, 594)

'Test set shape: '

(3746, 594)

,index,dt,shop_tag_15_rank,shop_tag_19_rank,shop_tag_2_rank,shop_tag_26_rank,shop_tag_34_rank,shop_tag_35_rank,shop_tag_36_rank,shop_tag_37_rank,shop_tag_45_rank,shop_tag_48_rank,shop_tag_50_rank,shop_tag_6_rank,shop_tag_15_rank_t-1,shop_tag_19_rank_t-1,shop_tag_2_rank_t-1,shop_tag_26_rank_t-1,shop_tag_34_rank_t-1,shop_tag_35_rank_t-1,shop_tag_36_rank_t-1,shop_tag_37_rank_t-1,shop_tag_45_rank_t-1,shop_tag_48_rank_t-1,shop_tag_50_rank_t-1,shop_tag_6_rank_t-1,shop_tag_15_rank_t-2,...,shop_tag_1_rank_t-9,shop_tag_1_rank_t-10,shop_tag_1_rank_t-11,shop_tag_5_rank,shop_tag_5_rank_t-1,shop_tag_5_rank_t-2,shop_tag_5_rank_t-3,shop_tag_5_rank_t-4,shop_tag_5_rank_t-5,shop_tag_5_rank_t-6,shop_tag_5_rank_t-7,shop_tag_5_rank_t-8,shop_tag_5_rank_t-9,shop_tag_5_rank_t-10,shop_tag_5_rank_t-11,shop_tag_27_rank,shop_tag_27_rank_t-1,shop_tag_27_rank_t-2,shop_tag_27_rank_t-3,shop_tag_27_rank_t-4,shop_tag_27_rank_t-5,shop_tag_27_rank_t-6,shop_tag_27_rank_t-7,shop_tag_27_rank_t-8,shop_tag_27_rank_t-9,shop_tag_27_rank_t-10,shop_tag_27_rank_t-11
10,21,22.0,1.0,1.0,1.0,50.0,1.0,1.0,48.0,49.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,48.0,50.0,1.0,49.0,47.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
11,22,23.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,49.0,48.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,48.0,49.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
22,21,22.0,1.0,1.0,1.0,50.0,1.0,1.0,48.0,49.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,48.0,50.0,1.0,49.0,47.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
23,22,23.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,49.0,48.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,48.0,49.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
34,21,22.0,47.0,1.0,1.0,1.0,1.0,1.0,44.0,48.0,46.0,1.0,45.0,1.0,48.0,1.0,47.0,1.0,1.0,1.0,44.0,1.0,45.0,43.0,46.0,1.0,46.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22451,22,23.0,46.0,1.0,1.0,1.0,1.0,1.0,48.0,47.0,45.0,50.0,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,43.0,45.0,41.0,47.0,49.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
22462,21,22.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47.0,45.0,49.0,1.0,1.0,1.0,1.0,1.0,1.0,46.0,1.0,1.0,45.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
22463,22,23.0,1.0,49.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
22474,21,22.0,47.0,49.0,50.0,41.0,1.0,1.0,1.0,45.0,44.0,1.0,1.0,1.0,45.0,43.0,49.0,1.0,1.0,1.0,1.0,48.0,40.0,1.0,1.0,1.0,49.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [25]:
x_train = np.array(train_df[rank_col])
y_train = np.array(train_df[y_col].loc[:,'top1'])
x_validation = np.array(test_df[rank_col])
y_validation = np.array(test_df[y_col].loc[:,'top1'])

In [26]:
clf = CatBoostClassifier(
    iterations=2000,
    random_seed=42,
    learning_rate=0.03,
    loss_function='MultiClass',
    use_best_model=True,
    task_type="GPU",
    devices='0:1'
)

clf.fit(
    x_train, y_train,
#     cat_features=cat_features,
    eval_set=(x_validation, y_validation),
    verbose=100
)

0:	learn: 3.6949884	test: 3.6965552	best: 3.6965552 (0)	total: 138ms	remaining: 4m 36s
100:	learn: 2.3087623	test: 2.4098267	best: 2.4098267 (100)	total: 8.93s	remaining: 2m 47s
200:	learn: 2.1432094	test: 2.3052780	best: 2.3052780 (200)	total: 17.1s	remaining: 2m 32s
300:	learn: 2.0368322	test: 2.2574908	best: 2.2574908 (300)	total: 25.4s	remaining: 2m 23s
400:	learn: 1.9595560	test: 2.2331815	best: 2.2331815 (400)	total: 33.4s	remaining: 2m 13s
500:	learn: 1.8965897	test: 2.2182654	best: 2.2182654 (500)	total: 41.2s	remaining: 2m 3s
600:	learn: 1.8425289	test: 2.2087451	best: 2.2087451 (600)	total: 49s	remaining: 1m 53s
700:	learn: 1.7950906	test: 2.2019722	best: 2.2019636 (699)	total: 56.4s	remaining: 1m 44s
800:	learn: 1.7506471	test: 2.1967862	best: 2.1967450 (799)	total: 1m 3s	remaining: 1m 35s
900:	learn: 1.7093947	test: 2.1935296	best: 2.1935051 (897)	total: 1m 11s	remaining: 1m 27s
1000:	learn: 1.6698193	test: 2.1902913	best: 2.1902657 (992)	total: 1m 18s	remaining: 1m 18s
110

In [27]:
display(clf.score(x_train, y_train))

0.5581420181526963

In [28]:
display(clf.score(x_validation, y_validation))

0.4257875066737854

In [29]:
print(y_train.tolist())

['37', '34', '34', '37', '37', '26', '37', '26', '35', '26', '37', '34', '34', '37', '37', '26', '37', '26', '35', '26', '46', '10', '10', '10', '10', '10', '10', '11', '10', '10', '37', '34', '37', '41', '36', '37', '37', '37', '37', '37', '17', '10', '17', '48', '48', '10', '19', '35', '10', '26', '10', '37', '10', '10', '2', '37', '10', '10', '10', '10', '12', '12', '12', '12', '12', '12', '12', '12', '12', '12', '45', '45', '45', '45', '46', '45', '45', '50', '50', '50', '37', '37', '37', '37', '36', '36', '36', '37', '37', '50', '17', '34', '10', '10', '10', '10', '45', '2', '2', '10', '37', '37', '37', '37', '37', '37', '37', '37', '48', '48', '18', '37', '2', '2', '18', '37', '25', '2', '2', '2', '13', '18', '10', '13', '13', '15', '24', '13', '13', '50', '50', '50', '50', '50', '50', '44', '50', '50', '50', '50', '6', '17', '15', '34', '19', '15', '6', '15', '34', '6', '37', '37', '37', '21', '21', '37', '15', '19', '37', '37', '10', '34', '10', '19', '19', '50', '16', '19', '5

In [30]:
clf.predict(x_train).tolist()

[['37'],
 ['34'],
 ['34'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['34'],
 ['34'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['17'],
 ['10'],
 ['16'],
 ['15'],
 ['48'],
 ['10'],
 ['10'],
 ['48'],
 ['10'],
 ['26'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['12'],
 ['45'],
 ['45'],
 ['45'],
 ['45'],
 ['45'],
 ['45'],
 ['45'],
 ['45'],
 ['45'],
 ['50'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['36'],
 ['36'],
 ['37'],
 ['37'],
 ['17'],
 ['34'],
 ['45'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['10'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['37'],
 ['18'],
 